# <b> Accréditation anonyme et coloriage de graphe </b>

<i> Compte-rendu du TP1 par Valentin SÉRÉE et Jérémy SERENA [INFO3]</i>

<h2> <u>Partie 1 :</u> Génération du graphe et du coloriage </h2>

In [ ]:
import random

# Cette fonction retourne une liste de n noeuds auxquels sont associées une couleur respective

def choixCouleur(n): 
    couleurs = ["rouge", "vert", "bleu"] # On définit toutes les couleurs disponibles
    couleursNoeuds=[] 
    for i in range (n):
        num=random.randint(0, 2) # On tire aléatoirement un entier entre 0 et 2
        couleursNoeuds.append(couleurs[num]) # On ajoute à la liste la couleur associée
    return couleursNoeuds

In [ ]:
import numpy as np

# Cette fonction retourne une matriche d'adjacence 
# Seuls les noeuds de couleurs différentes peuvent être reliés 
# Si deux noeuds sont "reliables", il y a une probabilité 1/2 pour qu'ils soient reliés

def genMatrice(tab) : # On fournit en entrée la liste des noeuds colorés
    matrice = np.empty((len(tab),len(tab)), dtype=str) # On crée un tableau de taille n²
    for i in range (0,len(tab)) :
        for j in range(i+1) :
            if tab[i] != tab[j] : # Si les couleurs sont différentes
                r = random.randint(0,1) # On définit si on relie ou non les deux
                if r==0 : # Si on décide de les relier
                    matrice[i][j]="x" # On utilise la symétrie
                    matrice[j][i]="x"
                else : # Si on décide de ne pas les relier
                    matrice[i][j]="." # On utilise la symétrie
                    matrice[j][i]="."
            else: # Si les couleurs sont identiques
                matrice[i][j]="." # On utilise la symétrie
                matrice[j][i]="."     
    return matrice

In [ ]:
# Cette fonction renvoie une matrice d'adjacence et le coloriage associé

def genererGraphe3Coloriable(n): # On fournit en entrée le nombre de noeuds souhaités
    coloriage = choixCouleur(n) # On crée un coloriage pour les n noeuds
    matrice = genMatrice(coloriage) # On crée la matrice associée 
    return(matrice, coloriage) 

In [ ]:
# DÉMONSTRATION : Test pour 20 noeuds

genererGraphe3Coloriable(20)

<h2> <u>Partie 2 :</u> Mise en gage des couleurs </h2>

In [ ]:
# Cette fonction crée une liste de n binaires de 128 bits

def creerRandom(n) : 
    t=[] # On crée la liste à renvoyer
    for i in range(n):
        valeur = ""
        for j in range (128):
            temp = str(random.randint(0,1)) # On tire un bit aléatoirement
            valeur += temp # On l'ajoute au nombre binaire (string)
        t.append(valeur)
    return t

In [ ]:
# Cette fonction permute les couleurs aléatoirement

def permutationCouleur(listeCouleurs): # On fournit en entrée la liste des noeuds colorés
    couleursBase = ["rouge","vert","bleu"] # On rappelle la liste de couleurs existantes
    couleursMelangees = [] # On crée la liste d'équivalence des couleurs précédentes
    listeMelangee = [] # On crée la liste à renvoyer 
    r1 = random.randint(0,2) 
    couleursMelangees.append(couleursBase[r1]) # On ajoute la couleur qui permute avec le rouge
    r2 = random.randint(0,2)
    while r1 == r2:
        r2 = random.randint(0,2)
    couleursMelangees.append(couleursBase[r2]) # On ajoute la couleur qui permute avec le vert
    r3 = 3-r2-r1
    couleursMelangees.append(couleursBase[r3]) # On ajoute la couleur qui permute avec le bleu
    for i in range (len(listeCouleurs)): # On remplit ensuite la liste permutée
        if (listeCouleurs[i] == "rouge"):
            listeMelangee.append(couleursMelangees[0])
        elif (listeCouleurs[i] == "vert"):
            listeMelangee.append(couleursMelangees[1])
        else: 
            listeMelangee.append(couleursMelangees[2])
    return (listeMelangee)

In [ ]:
# Cette fonction encode une couleur
# Une couleur fournie en entrée va se voir associer en sortie un codage en binaire

def encodageCouleur(couleur):
    if (couleur == "rouge"): 
        return "00" 
    elif (couleur == "vert"):
        return "01"
    else:
        return "10"

In [ ]:
# Cette fonction concatène le codage binaire de chaque noeud coloré au nombre binaire généré aléatoirement

def concatenation(listeCouleurs,listeRandom):
    hici = []
    for i in range (len(listeCouleurs)):
        hici.append(listeRandom[i] + encodageCouleur(listeCouleurs[i]))
    return hici

In [ ]:
import hashlib

# Cette fonction met en gage la liste de noeuds colorés 

def miseEnGage(listeCouleurs): # On fournit en entrée la liste des noeuds colorés 
    n = len(listeCouleurs) # On récupère le nombre de noeuds
    listeMelangee = permutationCouleur(listeCouleurs) # On permute les couleurs
    listeAleatoire = creerRandom(n) # On génère une liste de binaires aléatoires de 128 bits
    listeATransmettre = concatenation(listeMelangee, listeAleatoire) # On concatène (ri | ci)
    listeFinale = [] 
    for i in range (n):
        result = hashlib.sha1(listeATransmettre[i].encode()) # On hache grâce à SHA1
        listeFinale.append(result.hexdigest()) # On passe le résultat en hexadécimal
    return (listeMelangee, listeAleatoire, listeFinale)
# Sortie :
# listeMelangee = liste des couleurs des noeuds permutées
# listeAleatoire = liste des nombres binaires aléatoires de 128 bits associés à chaque noeud
# listeFinale = mise en gage 

In [ ]:
# DÉMONSTRATION : Test pour 20 noeuds

(mat, color) = genererGraphe3Coloriable(20) # On récupère la matrice d'adjacence (publique) et la liste des couleurs (secrète)
miseEnGage(color) # On récupère la mise en gage

<h2> <u>Partie 3 :</u> Preuve de connaissance à divulgation nulle d'un 3-coloriage </h2>

In [ ]:
# Cette fonction tire au sort 2 noeuds reliés de la matrice d'adjacence

def tirage(mat, n): # On fournit en entrée la matrice et le nombre de noeuds
    i = random.randint(0,n-1) # On tire un noeud i
    j = random.randint(0,n-1) # On tire un noeud j
    while (mat[i][j]!="x"): # Si les noeuds ne sont pas reliés, on recommence le tirage
        i = random.randint(0,n-1)
        j = random.randint(0,n-1)
    return (i,j) 

In [ ]:
# Cette fonction vérifie le coloriage pour deux noeuds reliés
    
def preuveColoriage(matAdj, miseGage, tabInfo): # On fournit en entrée la matrice d'adjacence, la mise en gage et les informations nécessaires pour ouvrir la mise en gage
    n = len(miseGage) # On récupère le nombre de noeuds
    coul = tabInfo[0] # On récupère les couleurs PERMUTÉES des noeuds
    rand = tabInfo[1] # On récupère les entiers de 128 bits
    (i,j) = tirage(matAdj, n) # On sélectionne 2 noeuds reliés
    if (coul[i] != coul[j]): # S'ils n'ont pas la même couleur
        biti = encodageCouleur(coul[i]) # On encode les couleurs en binaire
        bitj = encodageCouleur(coul[j])
        result1 = hashlib.sha1((rand[i] + biti).encode()).hexdigest() # On hache avec SHA1
        result2 = hashlib.sha1((rand[j] + bitj).encode()).hexdigest()
        if (result1 == miseGage[i] and result2 == miseGage[j]): # On vérifie si la mise en gage est correcte pour les deux noeuds
            return True
        else: # Si la mise en gage n'est pas égale
            return False
    else: # Si les noeuds sont de la même couleur
        return False

In [ ]:
# Cette fonction effectue le test 400 fois pour une matrice et une mise en gage

def preuve(matAdj, miseGage, tabInfo) :
    bool = True
    i = 400
    while bool and i!=0:
        bool = preuveColoriage(matAdj, miseGage, tabInfo)
        i-=1
    return bool

<h3> Scénario : montrer le fonctionnement des fonctions </h3>

In [ ]:
# On génère une matrice d'adjacence et un coloriage
(mat, color) = genererGraphe3Coloriable(20)

# La matrice (mat) est publique
# Le coloriage (color) est secret. 
# L'utilisateur veut ensuite prouver qu'il connait le coloriage


In [ ]:
##############
# CAS 1 
##############

# L'utilisateur 1 connaît le réel coloriage et va le mettre en gage
(colorMelang1, rand1, gage1) = miseEnGage(color) # il utilise "color", càd le vrai coloriage

# Il fournit au vérificateur la matrice connue (mat), sa mise en gage (gage1) et les informations (colorMelang1 et rand1)
preuve(mat,gage1,[colorMelang1, rand1]) # La vérification est vraie : l'utilisateur connait bien le coloriage

In [ ]:
##############
# CAS 2 
##############

# L'utilisateur 2 ne connaît pas le réel coloriage
# Il 'invente' donc un coloriage (ici, on utilise la fonction qui génère un coloriage aléatoire)
colorFake = choixCouleur(20)

# Il va le mettre en gage
(colorMelang2, rand2, gage2) = miseEnGage(colorFake)

# Il fournit au vérificateur ses données
preuve(mat,gage2,[colorMelang2, rand2]) # La vérification est fausse

In [ ]:
(mat, color) = genererGraphe3Coloriable(20)
(colorMelang, rand, gage) = miseEnGage(color)
preuve(mat,gage,[colorMelang, rand])

#écrire le scénario

<h2> <u>Partie 4 :</u> Analyse du protocole </h2>

<b> <u> Question 1 </u> </b>

Si un utilisateur obtient une solution valide, il peut la coder avec SHA1 après y avoir ajouté un nombre aléatoire. Lorsque le correcteur essaiera sa proposition, il pourra comparer à partir des couleurs et des hachés des différentes arêtes qu'il obtient les mêmes valeurs en refaisant ces calculs, et que les couleurs données sont différentes. Il prouve donc que la solution proposée valide la non adjacence de deux sommets de couleurs identiques. Ainsi, si les deux personnes effectuent le protocole rigoureusement, ils réalisent les mêmes opérations et le vérificateur compare l'authenticité de ce codage à son propre calcul.


<b> <u> Question 2 </u> </b>

Si un utilisateur essaie de proposer une fausse combinaison, il devra créer une combinaison qui sera suffisamment proche d'une combinaison valide pour tromper les tests effectués par le correcteur. Cependant, le grand nombre de tests (400) couplé au nombre d'arêtes rend la probabilité d'avoir une arrête fausse (entre deux noeuds de la même couleur) non testée très faible.

Cette probabilité vaut ((nb_arête-1)/nb_arête)^400. Par exemple, cette probabilité est égale à 3.1*10^-6 pour un nombre de 50 arêtes.

<b> <u> Question 3 </u> </b>

La décision de ne donner au correcteur que les hachés de la somme entre les couleurs (permutées !) et les entiers aléatoirement choisis empêche le correcteur de retrouver la valeur des couleurs initialement utilisées, il peut uniquement vérifier si deux couleurs sont identiques ou différentes, ce qui l'assure du respect de la consigne mais l'empêche de déterminer les couleurs utilisées. L'utilisateur ne fournit aucunement le coloriage initial au vérificateur.